In [1]:
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
import datetime

In [2]:
batch_size = 16
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_data(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
baseline_csv = "baseline.csv"
output_csv = "baselineVSmodel_predictions.csv"
model_path = "model_2024-11-26_21-46-02.pth" 
baseline_df = pd.read_csv(baseline_csv)

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', num_labels=8
)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

encodings = tokenize_data(baseline_df['Example Description'].tolist())
dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'])
data_loader = DataLoader(dataset, sampler=SequentialSampler(dataset), batch_size=batch_size)

predictions = []

model.eval()

with torch.no_grad():
    for batch in tqdm(data_loader, desc="Making Predictions"):
        input_ids, attention_mask = [item.to(device) for item in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().tolist())

baseline_df['model_2024-11-26_21-46-02 Prediction'] = [baseline_df['Artifact Id'].iloc[i] for i in predictions]

baseline_df['baseline Prediction'] = baseline_df['Prediction']

baseline_df[['Artifact Id', 'baseline Prediction', 'model_2024-11-26_21-46-02 Prediction']].to_csv(output_csv, index=False)
print(f"Predictions saved to {output_csv}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Making Predictions:   0%|          | 0/1 [00:00<?, ?it/s]

Predictions saved to baseline_predictions.csv
